In [1]:
import csv
import pandas as pd
import datetime as dt

In [2]:
L_parc_hist=[]
Parc_hist_df=pd.read_csv("Nucleaire_historique.csv",sep=';',decimal=',')
for i in Parc_hist_df.index:
    row=list(Parc_hist_df.loc[i])
    L_parc_hist.append(row)

In [3]:
L_new_nuc=[]
New_nuc_df=pd.read_csv("Nouveau_nucleaire.csv",sep=';',decimal=',')
for i in New_nuc_df.index:
    row=list(New_nuc_df.loc[i])
    L_new_nuc.append(row)

In [31]:
# Key functions
L_modul_year=[1.2,1.1,1,0.95,0.95,0.9,0.95,0.93,0.9,1,1.02,1.1]
taux_unav_hist=0.27
taux_unav_new=0.17
Power_EPR2=1670
Power_SMR=340

def duration_unav(row):
    if row[0]=='Flamanville-3':
        return round(365*taux_unav_new)
    else:
        return round(365*taux_unav_hist)

def complete_parc(L_parc_hist,L_new_nuc,year,nuclear_plus=True):
    L_parc_year=[]
    for row in L_parc_hist:
        if year<=row[2]:
            L_parc_year.append(row+[duration_unav(row),None,None])
    
    if year<L_new_nuc[0][0]:
        N_EPR2=0
        N_SMR=0
    elif year>=L_new_nuc[-1][0]:
        if nuclear_plus:
            N_EPR2=L_new_nuc[-1][1]
            N_SMR=L_new_nuc[-1][2]
        else:
            N_EPR2=L_new_nuc[-1][3]
            N_SMR=L_new_nuc[-1][4]
    else:
        if nuclear_plus:
            N_EPR2=L_new_nuc[year-L_new_nuc[0][0]][1]
            N_SMR=L_new_nuc[year-L_new_nuc[0][0]][2]
        else:
            N_EPR2=L_new_nuc[year-L_new_nuc[0][0]][3]
            N_SMR=L_new_nuc[year-L_new_nuc[0][0]][4]
    
    for i in range(N_EPR2):
        L_parc_year.append(['EPR2',Power_EPR2,2100]+[round(365*taux_unav_new),None,None])
    for i in range(N_SMR):
        L_parc_year.append(['SMR',Power_SMR,2100]+[round(365*taux_unav_new),None,None])
    
    return L_parc_year

L_month=[31,28,31,30,31,30,31,31,30,31,30,31]
def goal_month(L_parc_year,L_modul_year):
    P_moy=sum([x[1]*(365-x[3])/365 for x in L_parc_year])
    return [L_month[m]*P_moy*24*L_modul_year[m] for m in range(12)]

def prod_month(L_parc_year,m):# m in [0..11] here
    E=0
    d0=dt.datetime(2019,m+1,1)
    if m<11:
        d1=dt.datetime(2019,m+2,1)
    else:
        d1=dt.datetime(2020,1,1)
    for x in L_parc_year:
        if x[4]==None or x[4]>=d1 or x[5]<=d0:
            E+=x[1]*L_month[m]*24
        else:
            delta=min(d1,x[5])-max(d0,x[4])
            E+=(L_month[m]*24-(delta.days*24+delta.seconds/3600))*x[1]
    return E

d2020=dt.datetime(2020,1,1)
d1h=dt.timedelta(hours=1)
def optimize(L_parc_hist,L_new_nuc,L_modul_year,year,nuclear_plus=True,delta_plus=0.01,delta_minus=0.01):
    L_parc_year=complete_parc(L_parc_hist,L_new_nuc,year,nuclear_plus)
    L_goals=goal_month(L_parc_year,L_modul_year)
    for m in range(12):
        d0=dt.datetime(2019,m+1,1)
        if m<11:
            d1=dt.datetime(2019,m+2,1)
        else:
            d1=d2020
        E_prod=prod_month(L_parc_year,m)
        i=0
        while i<len(L_parc_year) and E_prod>(1+delta_plus)*L_goals[m]:
            if L_parc_year[i][4]!=None:
                i+=1
            else:
                delta_arret_tranche=dt.timedelta(days=L_parc_year[i][3])
                L_parc_year[i][4]=d0
                L_parc_year[i][5]=d0+delta_arret_tranche
                if L_parc_year[i][5]>d2020:
                    delta_depasse=L_parc_year[i][5]-d2020
                    L_parc_year[i][4]-=delta_depasse
                    L_parc_year[i][5]=d2020
                E_prod=prod_month(L_parc_year,m)
                while E_prod<(1-delta_minus)*L_goals[m] and L_parc_year[i][5]<d2020:
                    L_parc_year[i][4]+=d1h
                    L_parc_year[i][5]+=d1h
                    E_prod=prod_month(L_parc_year,m)
                i+=1
    return L_parc_year

def nuc_capacity(L_parc_year):
    P_old_nuc=0
    P_new_nuc=0
    for x in L_parc_year:
        if x[0] in ['EPR2','SMR']:
            P_new_nuc+=x[1]
        else:
            P_old_nuc+=x[1]
    return P_new_nuc, P_old_nuc

d2019=dt.datetime(2019,1,1)
def nuc_availability(L_parc_year):
    P_new_nuc, P_old_nuc=nuc_capacity(L_parc_year)
    
    L_dates=[]
    L_new_nuc=[0 for i in range(8760)]
    L_old_nuc=[0 for i in range(8760)]
    date=d2019
    for i in range(8760):
        L_dates.append(pd.Timestamp(date))
        date+=d1h
        for x in L_parc_year:
            if not x[4]<=L_dates[-1]<=x[5]:
                if x[0] in ['EPR2','SMR']:
                    L_new_nuc[i]+=x[1]
                else:
                    L_old_nuc[i]+=x[1]
                    
    L_new_nuc=[x/max(P_new_nuc,1) for x in L_new_nuc]
    L_old_nuc=[x/max(P_old_nuc,1) for x in L_old_nuc]
    
    return pd.DataFrame({"Date":L_dates,"New nuke":L_new_nuc,"Old nuke":L_old_nuc})
    

In [32]:
# Nuclear availability and capacity
L_capa_plus=[]
L_capa_minus=[]
for year in [2030,2040,2050,2060]:
    for nuclear_plus in [True,False]:
        L_parc_year=optimize(L_parc_hist,L_new_nuc,L_modul_year,year,nuclear_plus)
        print(L_parc_year)
        Availability_df=nuc_availability(L_parc_year)
        if nuclear_plus:
            char_plus="nuclear_plus"
            L_capa_plus.append(list(nuc_capacity(L_parc_year)))
        else:
            char_plus="nuclear_minus"
            L_capa_minus.append(list(nuc_capacity(L_parc_year)))
        Availability_df.to_csv("availability_nuclear_"+str(year)+"_"+char_plus+".csv",sep=";",decimal=".",index=False)
Capa_plus_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_plus],"Old nuke":[x[1] for x in L_capa_plus]})
Capa_minus_df=pd.DataFrame({"Year":[2030,2040,2050,2060],
                           "New nuke":[x[0] for x in L_capa_minus],"Old nuke":[x[1] for x in L_capa_minus]})
Capa_plus_df.to_csv("Capacity_nuclear_plus.csv",sep=";",decimal=".",index=False)
Capa_minus_df.to_csv("Capacity_nuclear_minus.csv",sep=";",decimal=".",index=False)

[['Bugey-4', 880, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-1', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-2', 915, 2030, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-3', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Tricastin-4', 915, 2035, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-1', 890, 2039, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-3', 890, 2040, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Dampierre-4', 890, 2040, 99, datetime.date

[['Bugey-5', 880, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-2', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-3', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Dampierre-4', 890, 2040, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-3', 910, 2041, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-4', 910, 2041, 99, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 4, 10, 0, 0)], ['Gravelines-5', 910, 2045, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Gravelines-6', 910, 2045, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['St-Laurent-B1', 915, 2042, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['St-Laurent-B2', 915, 2042, 99, d

[['Flamanville-3', 1650, 2083, 62, datetime.datetime(2019, 1, 1, 0, 0), datetime.datetime(2019, 3, 4, 0, 0)], ['Cattenom-3', 1300, 2051, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Cattenom-4', 1300, 2052, 99, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 5, 11, 0, 0)], ['Penly-1', 1330, 2050, 99, datetime.datetime(2019, 2, 4, 5, 0), datetime.datetime(2019, 5, 14, 5, 0)], ['Penly-2', 1330, 2052, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Golfech-1', 1310, 2051, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Golfech-2', 1310, 2053, 99, datetime.datetime(2019, 3, 1, 0, 0), datetime.datetime(2019, 6, 8, 0, 0)], ['Chooz-B1', 1500, 2056, 99, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 7, 9, 0, 0)], ['Chooz-B2', 1500, 2057, 99, datetime.datetime(2019, 4, 14, 3, 0), datetime.datetime(2019, 7, 22, 3, 0)], ['Civaux-1', 1495, 2058, 99, datetime.datetime(20

[['Flamanville-3', 1650, 2083, 62, datetime.datetime(2019, 2, 1, 0, 0), datetime.datetime(2019, 4, 4, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 3, 1, 23, 0), datetime.datetime(2019, 5, 2, 23, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 1, 0, 0), datetime.datetime(2019, 6, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 4, 21, 17, 0), datetime.datetime(2019, 6, 22, 17, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 5, 21, 7, 0), datetime.datetime(2019, 7, 22, 7, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 6, 1, 0, 0), datetime.datetime(2019, 8, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 7, 9, 15, 0), datetime.datetime(2019, 9, 9, 15, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 8, 1, 0, 0), datetime.datetime(2019, 10, 2, 0, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 8, 14, 18, 0), datetime.datetime(2019, 10, 15, 18, 0)], ['EPR2', 1670, 2100, 62, datetime.datetime(2019, 9, 13, 20, 0), datetime.d